## Tensorflow + RNN 을 활용한 영화 리뷰 감성 분석

#### (1) 드라이브 마운트 및 mecab 다운로드

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! wget --no-check-certificate https://raw.github.com/SOMJANG/Mecab-ko-for-Google-Colab/master/install_mecab-ko_on_colab190912.sh
!bash install_mecab-ko_on_colab190912.sh

#### (2) 필요 모듈 임포트, 전처리 된 데이터 로드

In [0]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant
from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
import numpy as np
import pickle
import os

tf.set_random_seed(1109)
mecab = Mecab()

with open("/content/drive/My Drive/Colab Notebooks/preprocessed_data.pkl", "rb") as f:
  saved_data = pickle.load(f)
  
word2idx = saved_data["word2idx"]
embedding_matrix = saved_data["embedding_matrix"]

train_sents = saved_data["train_sents"]
test_sents = saved_data["test_sents"]

train_labels = saved_data["train_labels"]
test_labels = saved_data["test_labels"]

# train_data 에서 랜덤하게 10% 를 검증 데이터로 구성
train_sents, val_sents, train_labels, val_labels = train_test_split(train_sents, train_labels, test_size=0.1, random_state=1109)

#### (2) 토크나이저 함수 정의

In [0]:
def tokenizer(sent):
  res = []
  tmp = mecab.pos(sent)
  tmp = list(map(lambda x: x[0]+"/"+x[1], tmp))
  for i, word in enumerate(tmp):
    idx = word2idx.get(word)
    if idx != None:
      res.append(idx)
    else:
      res.append(word2idx.get("<UNK>")) # out of vocab word 처리

  return np.array([res], dtype="int32")

#### (3) 인풋 데이터 형태로 만들기

In [0]:
# train_sents, val_sents 를 word2idx 를 사용해서 각 단어별 index 값을 가지는 numpy array 형태로 구축
# 전체 데이터에 대해서 train_data 의 max_length 기준으로 padding 진행

max_length = max([len(sent) for sent in train_sents])
train_seqs = []
val_seqs = []
test_seqs = []

for sent in train_sents:
  tmp = np.zeros(max_length, dtype="int32")
  for i, word in enumerate(sent):
    idx = word2idx.get(word)
    if idx != None:
      tmp[i] = idx
    else:
      tmp[i] = word2idx.get("<UNK>") # out of vocab word 처리
  train_seqs.append(tmp)

for sent in val_sents:
  tmp = np.zeros(max_length, dtype="int32")
  for i, word in enumerate(sent):
    idx = word2idx.get(word)
    if idx != None:
      tmp[i] = idx
    else:
      tmp[i] = word2idx.get("<UNK>")
  val_seqs.append(tmp)

for sent in test_sents:
  tmp = np.zeros(max_length, dtype="int32")
  for i, word in enumerate(sent):
    idx = word2idx.get(word)
    if idx != None:
      tmp[i] = idx
    else:
      tmp[i] = word2idx.get("<UNK>")
  test_seqs.append(tmp)


train_inputs = np.stack(train_seqs)
val_inputs = np.stack(val_seqs)
test_inputs = np.stack(test_seqs)

train_targets = np.array(train_labels, dtype="int32")
val_targets = np.array(val_labels, dtype="int32")
test_targets = np.array(test_labels, dtype="int32")

#### (4) 텐서플로우 모델 구축하기

* 하이퍼파라미터는 자유롭게 설정
* CuDNNGRU 사용, 3-layer stacked GRU, Bidirectional 사용
* val_loss 기준으로 가장 성능이 좋은 모델을 best_model_GRU.h5 로 저장

#### -- (i) 하이퍼파라미터 설정

In [0]:
# 자유롭게 하이퍼파라미터를 수정해주세요~!!
epochs = 10
batch_size = 1024
learning_rate = 0.01
hidden_size = 32
dropout = 0.3

vocab_size = len(word2idx) # don't change
embedding_size = 100 # don't change
max_length = train_inputs.shape[1] # don't change

#### -- (ii) 모델 정의

In [0]:
model = tf.keras.Sequential()
model.add(layers.Embedding(input_dim = vocab_size,
                           output_dim = embedding_size,
                           embeddings_initializer = Constant(embedding_matrix),
                           trainable = False))

# your code here
# stacked RNN 사용시, 가장 마지막 layer 를 제외하고는 return sequences 를 입력해주어야 함




model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

#### -- (iii) 모델 학습 및 저장

In [0]:
adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=adam,
              loss="binary_crossentropy",
              metrics=["accuracy"])

saved_path = "/content/drive/My Drive/Colab Notebooks/best_model_GRU.h5"
saved_dir = os.path.dirname(saved_path)

# 가장 성능이 좋은 모델을 저장
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    saved_path, verbose=1, save_best_only=True, monitor="val_loss")


# your code here - fit() 안의 인자를 채워주세요 !!
model.fit()

#### (5) 학습된 모델 불러오기

In [0]:
# your code here - pretrained_model 변수에 기존에 학습시킨 모델을 불러와서 넣어주세요 !!
pretrained_model = 
pretrained_model.summary()

#### (6) 테스트 데이터 평가

In [0]:
# your code here - pretrained_model.evaluate() 함수를 사용해서 테스트 데이터데 대한 평가를 진행해주세요.
# pretrained_model.evaluate() 안의 인자를 채워주세요

test_batch_size = 512
pretrained_model.evaluate()

#### (7) 내가 만든 문장으로 결과 확인하기

In [0]:
# your code here - pos_sample 안에 내가 만든 긍정 문장을 넣어서 결과를 확인해보세요 !!

pos_sample = ""
pretrained_model.predict(tokenizer(pos_sample))

In [0]:
# your code here - neg_sample 안에 내가 만든 부정 문장을 넣어서 결과를 확인해보세요 !!

neg_sample = ""
pretrained_model.predict(tokenizer(neg_sample))